In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error

# Load the dataset
data = pd.read_csv('dataset/AEP_hourly.csv')

# Preprocess the data with the correct datetime format
# Preprocess the data with the correct datetime format
data['Datetime'] = pd.to_datetime(data['Datetime'], format='%Y-%m-%d %H:%M:%S')
data.set_index('Datetime', inplace=True)
data.dropna(inplace=True)

# Split the data into train and test sets
train_size = int(len(data) * 0.8)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

# Fit ARIMA model to train data
arima_model = ARIMA(train_data['AEP_MW'], order=(2, 1, 2))
arima_result = arima_model.fit()

# Make predictions with ARIMA on test data
arima_forecast = arima_result.forecast(steps=len(test_data))

# Ensure compatible indices for test_data and arima_forecast
arima_forecast.index = test_data.index

# Compute residuals (differences between actual and ARIMA forecasts)
residuals = test_data['AEP_MW'] - arima_forecast

# Scale residuals using Min-Max scaling
scaler = MinMaxScaler()
residuals_scaled = scaler.fit_transform(np.array(residuals).reshape(-1, 1))

# Prepare input-output pairs for ANN
seq_length = 24  # Sequence length for input data
X, y = [], []
for i in range(len(residuals_scaled) - seq_length):
    X.append(residuals_scaled[i:i + seq_length])
    y.append(residuals_scaled[i + seq_length])
X, y = np.array(X), np.array(y)

# Split data into train and test sets for ANN
train_size_ann = int(len(X) * 0.8)
X_train, X_test = X[:train_size_ann], X[train_size_ann:]
y_train, y_test = y[:train_size_ann], y[train_size_ann:]

# Define and train the ANN model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(seq_length, 1)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, verbose=1)

# Make predictions with the trained ANN
y_pred = model.predict(X_test)

# Inverse transform predictions and actual values
y_pred_inv = scaler.inverse_transform(y_pred.reshape(-1, 1))
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Calculate RMSE (Root Mean Squared Error)
rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
print(f"Test RMSE: {rmse}")

# Plot training and validation loss of ANN
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('ANN Training and Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

# Plot actual vs. predicted values on the test set
plt.figure(figsize=(12, 6))
plt.plot(test_data.index[train_size + seq_length:], test_data['AEP_MW'][train_size + seq_length:], label='Actual Data', marker='o')
plt.plot(test_data.index[train_size + seq_length:], arima_forecast, label='ARIMA Forecast', linestyle='--')
plt.plot(test_data.index[train_size + seq_length:], arima_forecast + y_pred_inv, label='Hybrid Model Forecast', linestyle='--')
plt.xlabel('Datetime')
plt.ylabel('AEP MW')
plt.title('Hybrid Model Forecasting: Actual vs. Predicted')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

# Function to calculate model accuracy metrics
def calculate_accuracy(y_true, y_pred):
    mae = np.mean(np.abs(y_true - y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r_squared = 1 - (np.sum((y_true - y_pred)**2) / np.sum((y_true - np.mean(y_true))**2))
    return mae, mape, r_squared

# Calculate accuracy metrics for the test set
mae, mape, r_squared = calculate_accuracy(y_test_inv, arima_forecast + y_pred_inv)

print(f"Test MAE: {mae:.2f}")
print(f"Test MAPE: {mape:.2f}%")
print(f"Test R-squared: {r_squared:.4f}")


C:\Users\mobiz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mobiz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it is not monotonic and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\mobiz\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._

Epoch 1/50
546/546 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.0310 - val_loss: 0.0150
Epoch 2/50
546/546 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0217 - val_loss: 0.0146
Epoch 3/50
546/546 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0211 - val_loss: 0.0145
Epoch 4/50
546/546 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0213 - val_loss: 0.0144
Epoch 5/50
546/546 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0212 - val_loss: 0.0143
Epoch 6/50
546/546 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0216 - val_loss: 0.0148
Epoch 7/50
546/546 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0214 - val_loss: 0.0146
Epoch 8/50
546/546 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0217 - val_loss: 0.0144
Epoch 9/50
546/546 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0214 - val_loss: 0.0142
Epoch 10/50
546/546 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0215 - val_loss: 0.0148
Epoch 11/50
546/546 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0213 - val_loss: 0.0148
Epoch 12/50
546/546 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

ValueError: Found input variables with inconsistent numbers of samples: [4847, 116328]